In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv("data/ratings_train.txt", sep="\t")
test_data = pd.read_csv("data/ratings_test.txt", sep="\t")

In [3]:
#데이터확인
train_data.info(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [4]:
#데이터확인
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [5]:
# 결측치 (missing value) 처리
na_mask = train_data['document'].isna()
na_removed_train = train_data[~na_mask].copy()
filtered_train = train_data[~na_mask].copy() # ~은 not & 은 그리고 | 는 또는

na_mask = test_data['document'].isna()
na_removed_test = test_data[~na_mask].copy() # ~은 not & 은 그리고 | 는 또는
filtered_test = test_data[~na_mask].copy()
na_removed_test


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [6]:
import re
filtered_train['document'] = na_removed_train['document'].map(lambda v : re.sub("[^A-Za-zㄱ-힝]", ' ', v))
filtered_test['document'] = na_removed_test['document'].map(lambda v : re.sub("[^A-Za-zㄱ-힝]", ' ', v))

In [7]:
from konlpy.tag import Okt # 한국어 사전 준비 ( 형태소 분할에 사용 )

okt = Okt()
print(okt.morphs(filtered_train['document'][0]))

['아', '더빙', '진짜', '짜증나네요', '목소리']


In [1]:
# ratings_na_removed_train.to_csv("data-files/ratings_train2.csv", index=False)
# ratings_na_removed_test.to_csv("data-files/ratings_test2.csv", index=False)

In [53]:
#데이터 다시 읽어오기
ratings_test2 = pd.read_csv('data/ratings_test2.csv')
ratings_train2 = pd.read_csv('data/ratings_train2.csv')

In [54]:
# document에 리스트 형식을 그대로 문자열로 저장되있음 -> 리스트로 다시 변경해줘야함
ratings_test2.head(2)

,id,document,label
0,6270596,"['굳', 'ㅋ']",1
1,9274899,['GDNTOPCLASSINTHECLUB'],0


In [55]:
#documnet column 자료형 변경 : string -> list
#print("['123', '456']"[3])
#print(['123', '456'][1])
#print(eval("['123', '456', '789']")[1])

ratings_test2['document'] = ratings_test2['document'].map(eval)
ratings_train2['document'] = ratings_train2['document'].map(eval)

In [56]:
type(ratings_test2['document'][0]), type(ratings_train2['document'][0])

(list, list)

In [57]:
# 토큰리스트 -> 단일 문자열
ratings_train2['document'] = ratings_train2['document'].str.join(' ')
ratings_test2['document'] = ratings_test2['document'].str.join(' ')

In [58]:
ratings_test2['document'][2]

'뭐 야 이 평점 들 은 나쁘진 않지만 점 짜 리 는 더 더욱 아니잖아'

In [59]:
#전처리 결과 파일에 저장
#ratings_test2.to_csv('data/ratings_test3.csv', index=False)
#ratings_train2.to_csv('data/ratings_train3.csv', index=False)

In [72]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

cv = CountVectorizer()
cv_train_counts = cv.fit_transform(ratings_train2['document'])
cv_test_counts = cv.transform(ratings_test2['document'])

tv = TfidfVectorizer()
tv_train_counts = tv.fit_transform(ratings_train2['document'])
tv_test_counts = tv.transform(ratings_test2['document'])

tt = TfidfTransformer()
tt_train_counts = tt.fit_transform(cv_train_counts)
tt_test_counts = tt.transform(cv_test_counts)

In [61]:
print(tv_test_counts.data)
print(tv_test_counts.indices)
print(tv_test_counts.indptr)

[0.53689267 0.41640707 0.51987093 ... 0.48321544 0.70766944 0.70654367]
[14256 22489 56621 ... 94077 29914 68995]
[     0      0      0 ... 391491 391501 391503]


In [79]:
ratings_test2['document'][2]

'뭐 야 이 평점 들 은 나쁘진 않지만 점 짜 리 는 더 더욱 아니잖아'

In [88]:
idx_to_word = { idx: w for w, idx in cv.vocabulary_.items() }

In [93]:
idx_to_word[14256], idx_to_word[22489], idx_to_word[56621], idx_to_word[58688], idx_to_word[90697]

('나쁘진', '더욱', '아니잖아', '않지만', '평점')

In [74]:
print(cv_test_counts.min())
print(cv_test_counts.max())
print(cv_test_counts)

0
35
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 391503 stored elements and shape (49997, 98323)>
  Coords	Values
  (2, 14256)	1
  (2, 22489)	1
  (2, 56621)	1
  (2, 58688)	1
  (2, 90697)	1
  (3, 30264)	1
  (3, 41065)	1
  (3, 58632)	1
  (3, 62424)	1
  (3, 65203)	1
  (3, 78757)	1
  (3, 80766)	1
  (4, 14828)	1
  (4, 20433)	1
  (4, 44594)	1
  (4, 54296)	1
  (4, 56570)	1
  (4, 80103)	1
  (4, 92974)	1
  (5, 63674)	1
  (5, 68381)	2
  (5, 78721)	1
  (5, 85531)	1
  (6, 55730)	1
  (6, 81695)	1
  :	:
  (49993, 70734)	1
  (49993, 79101)	1
  (49993, 88270)	1
  (49993, 92331)	1
  (49994, 9947)	1
  (49994, 16586)	1
  (49994, 18575)	1
  (49994, 23030)	1
  (49994, 30585)	1
  (49994, 41221)	1
  (49994, 44484)	1
  (49994, 65187)	1
  (49994, 78108)	1
  (49995, 10833)	1
  (49995, 49246)	1
  (49995, 50437)	1
  (49995, 61839)	1
  (49995, 62493)	1
  (49995, 63674)	1
  (49995, 74225)	1
  (49995, 76372)	1
  (49995, 86132)	1
  (49995, 94077)	1
  (49996, 29914)	1
  (49996, 68995)	1


In [68]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=10, max_iter= 1000)
logreg.fit(tv_train_counts, ratings_train2['label'])
print(logreg.score(tv_train_counts, ratings_train2['label']))
print(logreg.score(tv_test_counts, ratings_test2['label']))

0.9356578552618421
0.8369102146128767


In [100]:
ratings_train2['document'][1]

'흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나'

In [104]:
num_list = [3546,40836,58351,63115,63674,64282,77609,85316,90822]
print(' '.join([idx_to_word[num] for num in num_list]))

가볍지 보고 않구나 연기 영화 오버 조차 초딩 포스터


In [95]:
print(cv_train_counts.min())
print(cv_train_counts.max())
print(cv_train_counts)

0
64
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1225586 stored elements and shape (149995, 98323)>
  Coords	Values
  (0, 22467)	1
  (0, 34543)	1
  (0, 81759)	1
  (0, 82398)	1
  (1, 3546)	1
  (1, 40836)	1
  (1, 58351)	1
  (1, 63115)	1
  (1, 63674)	1
  (1, 64282)	1
  (1, 77609)	1
  (1, 85316)	1
  (1, 90822)	1
  (2, 20032)	1
  (2, 28840)	1
  (2, 35573)	1
  (2, 37295)	1
  (2, 41258)	1
  (2, 85789)	1
  (3, 8472)	1
  (3, 8627)	1
  (3, 51139)	1
  (3, 61951)	1
  (3, 69779)	1
  (3, 74225)	1
  :	:
  (149990, 35780)	1
  (149990, 70421)	1
  (149990, 78694)	1
  (149991, 16360)	1
  (149991, 17168)	1
  (149991, 90697)	1
  (149992, 5846)	1
  (149992, 32988)	1
  (149992, 83849)	1
  (149992, 91817)	1
  (149992, 93192)	1
  (149992, 96154)	1
  (149993, 39102)	1
  (149993, 63674)	1
  (149993, 66187)	1
  (149993, 72822)	1
  (149993, 85104)	1
  (149993, 85535)	1
  (149994, 16959)	1
  (149994, 21265)	1
  (149994, 51293)	1
  (149994, 63674)	2
  (149994, 85655)	1
  (149994, 92312)	1
  (149994, 